# PART1

## Json import to openSearch

In [1]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

## Login to openSearch

In [ ]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]

## Create index structure

In [ ]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

## Choose the tokenizer and model that will be used in the embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json", tokenizer, model).createMap()

In [ ]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()


In [ ]:
import search as s

s.Search(client, index_name, tokenizer, model).queryOpenSearch('Holiday Salad', 5,None, None, ["salads"], ["lupine"], None)

## Testing

In [ ]:
import searchJson as sj
metrics = sj.SearchJson(client, index_name, tokenizer, model)
metrics.searchJson("../jsonData/cleanAnnotations.json")

In [ ]:
delete this sentence to run
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)

# PART2

## Compute the step similarity matrix

### Compute the steps json files

In [1]:
import stepsCalculator as sc

sc.CreateStepsMaps("../jsonData/recipesMap.json", '../jsonData/stepsImage.json', '../jsonData/stepsNoImage.json').createMaps()


### Compute the embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
import computeImagesEmbeddings as cie

cie.ComputeImagesEmbeddings("../jsonData/stepsImage.json","../jsonData/stepsNoImage.json","../jsonData/stepsImageEmbeddingsMap.json","../jsonData/stepsNoImageEmbeddingsMap.json", tokenizer, model).createMap()

### Compute the similarity matrix

In [1]:
import computeMatrix as cm

imageSimilarityMatrix = cm.ComputeMatrix("../jsonData/stepsImageEmbeddingsMap.json", "../jsonData/stepsNoImageEmbeddingsMap.json").createMatrix()
print(imageSimilarityMatrix)
print(len(imageSimilarityMatrix[0]))

[[ 0.18038417  0.32493275  0.09035419 ...  0.24216047  0.1828222
   0.06139156]
 [ 0.22687672  0.3896319   0.36241928 ...  0.33001146  0.29410326
   0.21193336]
 [ 0.2823929   0.29953444  0.2362206  ...  0.2590037   0.15724806
   0.09927286]
 ...
 [ 0.18877986  0.06847328  0.1471263  ...  0.18271409  0.20773083
   0.1412945 ]
 [-0.03327034 -0.04340815  0.10297828 ... -0.08764414 -0.04812049
  -0.03889523]
 [ 0.21104099  0.1107926   0.2918464  ...  0.3544004   0.27908713
   0.1592933 ]]
804


In [2]:
import computeClipOutput as cco

cco.ComputeClipOutput(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/").getImage()

ValueError: operands could not be broadcast together with shapes (4,224,224) (3,) 